This notebook presents an application semi-physical model of the L1 laser in its second version, based on PyTorch.
In particular, in this notebook the optimization process of the control parameters is solved using such a model. 

Author: Francesco Capuano, 2022 S17 summer intern @ ELI-beamlines, Prague


# Motivation

The goal of this project is to maximise second-harmonic efficiency. However, since this metric is also very much related to the shortest possible pulse shape, we started with developing a strategy to optimise a predefinite set of control parameters so as to minimise the difference between the obtained pulse shape (in the temporal domain) and a target one (which, by default, is the shortest one typically). 

However, since data are really expensive to empirically collect we resorted to model the underlying dynamics of the whole system, also considering that (even if not exhaustive) there is a significant amount of know-how concerned with the considered dynamics available.

After this model is obtained, it is possible to use it to obtain the desired control parameters.

In [1]:
import torch
# these import are necessary to import modules from directories one level back in the folder structure
import sys
sys.path.append("../..")
from utils.se import get_project_root
from algorithms.L1_BayesianOptimisation import extract_data
import matplotlib.pyplot as plt
from scipy.optimize import Bounds
import numpy as np

frequency, field = extract_data()

The preprocessing steps do not depend on the control parameters, therefore they can take place even in numpy

In [2]:
# preprocessing steps
from utils.physics import *
# preprocessing
cutoff = np.array((289.95, 291.91)) * 1e12
# cutting off the signal
frequency_clean, field_clean = cutoff_signal(frequency_cutoff = cutoff, frequency = frequency * 1e12,
                                             signal = field)
# augmenting the signal
frequency_clean_aug, field_clean_aug = equidistant_points(frequency = frequency_clean,
                                                          signal = field_clean,
                                                          num_points = int(3e3)) # n_points defaults to 5e3
# retrieving central carrier
central_carrier = central_frequency(frequency = frequency_clean_aug, signal = field_clean_aug)

However, to be used in the Computational Laser model, their tensor version is required

In [3]:
from utils.LaserModel_torch import ComputationalLaser as CL

intensity = torch.from_numpy(field ** 2)
frequency, field = torch.from_numpy(frequency_clean_aug), torch.from_numpy(field_clean_aug)
compressor_params = -1 * torch.tensor([267.422 * 1e-24, -2.384 * 1e-36, 9.54893 * 1e-50], dtype = torch.double)

laser = CL(frequency = frequency * 1e-12, field = field, compressor_params = compressor_params)
target_time, target_profile = laser.transform_limited()

In [4]:
def L1Loss(GDD, TOD): 
    """
    This function computes the L1-Loss corresponding to a given value of GDD and TOD.
    """
    control = torch.stack([GDD,TOD,torch.tensor(9.54893 * 1e-50)]).requires_grad_(True)
    _, controlled_profile = laser.forward_pass(control)
    return (torch.abs(controlled_profile - target_profile)).sum()

In [5]:
from scipy.optimize import Bounds
bounds_control = Bounds(
                    # GDD         # TOD          # FOD
            lb = (2.3522e-22, -1.003635e-34, 4.774465e-50),
            ub = (2.99624e-22, 9.55955e-35, 1.4323395e-49)
        )

first_two_lower, first_two_upper = bounds_control.lb[:2], bounds_control.ub[:2]
bounds = torch.from_numpy(np.vstack((first_two_lower, first_two_upper)).T)

nsteps = 1000
GDD_linspace = torch.linspace(start = bounds[0,0], end = bounds[0,1], steps = nsteps)
TOD_linspace = torch.linspace(start = bounds[1,0], end = bounds[1,1], steps = nsteps)
GDD_mesh, TOD_mesh = torch.meshgrid(GDD_linspace, TOD_linspace, indexing = "ij")

TL_grid = torch.empty(size = (nsteps, nsteps))

In [14]:
# populating the mesh grid with values of L1 Loss per each combination of GDD and TOD considered
combinations = []
for i in tqdm(range(len(GDD_linspace))): 
    gdd = GDD_linspace[i]
    for j in range(len(TOD_linspace)): 
        tod = TOD_linspace[j]
        TL_grid[i,j] = L1Loss(gdd, tod)
        combinations.append((gdd, tod))

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 4601.29it/s]


The objective function could also be plotted when considered as a function of **GDD** and **TOD** only.

In [ ]:
my_map = "plasma"
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.set_xlabel(r"$GDD$"); ax.set_ylabel(r"$TOD$"); ax.set_zlabel("L1-Norm of Error")

surf = ax.plot_surface(GDD_mesh.numpy(), TOD_mesh.numpy(), TL_grid.numpy(), cmap = my_map)

ax.set_title("Objective Function Landscape", fontweight = "bold")
fig.colorbar(surf, shrink = 0.5)

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots()

cset = ax.contourf(
    GDD_mesh.numpy(), 
    TOD_mesh.numpy(), 
    TL_grid.numpy(), 
    offset = TL_grid.numpy().min(),
    cmap = my_map
)

fig.colorbar(cset)

In [ ]:
def adagrad(
    f,
    x:torch.tensor,
    eta_0:float=1.,
    maxit:int=int(1e3),
    tolgrad:float=1e-6,
    eps:float=1e-6)->Tuple[torch.tensor, torch.tensor]:
    
    """
    This function implements ADAGRAD optimisation algorithm.
    """
    k = 0
    xk = x
    # giving gradient to xk
    f(xk).backward()
    
    points = torch.zeros(size = (maxit, len(x)))
    grads = torch.zeros(maxit)
    
    grad_k = xk.grad
    
    sum_grad_squared = grad_k ** 2
    lr = eta_0 / torch.sqrt(eps + sum_grad_squared)
    
    for k in (pbar := tqdm(range(maxit))): 
        # store & display information
        points[k, :] = xk.detach()
        grads[k] = norm(grad_k)
        pbar.set_description("It. {} - Norm Grad {:4e}".format(k+1, norm(grad_k)))
        # update position      
        xk = (xk - lr * grad_k).detach().requires_grad_(True)
        # compute gradient at new position
        f(xk).backward()
        grad_k = xk.grad
        # adapt learning rate
        sum_grad_squared += grad_k ** 2
        lr = eta_0 / torch.sqrt(eps + sum_grad_squared)
        # reiterate (if not converged)
        if norm(grad_k) <= tolgrad: 
            points = points[:, :k]
            grads = grads[:k]
            break
        else: 
            k += 1
            continue